<a href="https://colab.research.google.com/github/hmelberg/health-analytics-using-python/blob/master/incidence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Estimating incidence of a disease based on patient registry data: Dealing with lookback and lookahead bias 

# Hans Olav Melberg, University of Oslo, 21. October, 2020


### Lookback bias
A key limitation when estimating disease incidence using data from health registries is the lack of records for the disease of interest before the earliest date of data availability. A patient who appears to be an incident patient in a given year, may have received the diagnosis before the registry started to capture data. Such misclassification of prevalent as incident cases leads to overestimation of incidence early in the observation period. This problem is sometimes labeled lookback bias or washout bias.


### Lookahead bias
The wash-out challenge interacts with another issue which may be labelled the lookahead problem. In order to avoid errors, studies of incidence based on health registry data sometimes require patients to have at least two or more registered events with the diagnostic code before classifying patients as having a given disease. This requirement creates a problem because patients towards the end of the observation period have had less time to accumulate multiple events and as a result there is an underestimation of incidence. 


### Consequence: Misleading estimate of trend
Taken together the underestimation of incidence in the beginning of the time period due to wash-out bias, and an underestimation towards the end caused by look-ahead bias, may create a false impressison that there is a declining trend in the incidence for the disease when the trend may be stable or even increasing. 

### Solution
One soution would be to examine the importance of lookback and lookahead using within the available data and to find a function that describes the relationship between the length of the lookback/lookahed time period and how it affects incidence. This function can be used to estimate what the incidence would be with longer lookback/lookahead periods.

More specifically, the method uses the observed dataset to estimate a non-linear function indicating the degree to which patients will remain incidence patients as more data is added before and after a given year. Based on the regression results, we calculated bias-adjusted incidences i.e. the incidence one would expect with a very long observation-period. 

## Overview of functions
To perform the calculations, the following functions are useful:
- *select_rows()*
  - Select only the relevant rows i.e. those with the diganostic code that is relevant for the disease 
  - Useful if the dataframe consists of events with codes that are irrelevant for the disease you want to estimate

- *raw_incidence()* 
  - Counts the number of new cases each year that have no previous events in the database
  - Used as a benchmark - compare it with the adjusted results

- *lookback_pattern()*
  - The number (or percent) of new cases in a given year that remain new cases as the length of the look-back period is gradually increased
  - Used as datapoints to identify the underlying curve 

- *fit_curve()*  
  - Estimates the coefficient in a (possibly non-linear) function that gives the best fit to the pattern in the data
  - Uses the data produced by the _lookback_pattern()_ function (and the _single_pattern()_ function)
  - Implemented using scipy _optimize_ algorithm (Thank you, scipy team!)

- *single_pattern()*
  - The share of all patients in a year that are single event patients (with only one relevant registered event) and that remain single when more data is gradually added before and after the year in question
  - Used to adjust for lookahead bias when two events are required since it allows us to estimate how many cases one expects to be truly single (meaning and the rest will have two events)

- *adjusted_incidence()*
  - The number of new cases each year after adjusting for lookback and lookahead



In [ ]:
# import packages
import pandas as pd
import numpy as np
import scipy
from scipy import optimize
from datetime import datetime, timedelta


In [ ]:
def select_rows(df, codes, cols):
  """
  Marks all rows in a dafaframe that have the code(s) in one or some it is column(s)

  Args:
    df (dataframe): the dataframe with the codes and the columns
    codes (string or list of str): the codes to be marked
    cols (str or list of str): the colum(s) where the codes are
  
  Returns:
    A boolean series
  
  Note:
    Missing value is here the same as False (no code is found)
    The columns can have be single valued (with only one code in each cell) or have multiple values in each cell (one column only)
  """
    if not isinstance(codes, list):
        codes=[codes]
    
    # multiple columns with code(s)
    if isinstance(cols, list):
      rows=df[cols].isin(codes).any(axis=1)
    
    # one column with code(s)
    else:
        codes='|'.join(codes)
        rows=df[cols].str.contains(codes, na=False)
    return rows

In [ ]:
def raw_incidence(df, codes=None, cols=None, pid='pid', date='date', 
                  required_events=1):
  """
  Counts the number of new cases each year i.e. persons that have no previous events in the dataframe

  Args:
    df: dataframe with information about individual level events
    codes (string or list of str): the codes for the disease you want to count
    cols (str or list of str): the colum(s) where the codes are
    pid (str): the personal identifier used in the dataframe
    date (str): the column with the dates of the events
    required_events (int): the number of events required before a case is counted as having the disease
  
  Returns: 
    Series with the number of new cases for each year available in the dataframe
  """
    # if no codes are specified, all observations are relevant 
    # (when the dataframe already has only relevant observations)
    if codes:
        rows=select_rows(df=df, codes=codes, cols=cols, pid=pid)
        df=df[rows]    
              
    # identify first event
    first_event=df.groupby(pid)[date].min()
 
    if required_events>1:
        events=df.groupby(pid).size()
        enough_events = events>=required_events
        first_event=first_event[enough_events]
        
    # unadjusted incidence (number of cases)
    raw_incidence=first_event.dt.year.value_counts().sort_index()
    raw_incidence.index=raw_incidence.index.astype(int)
        
    return raw_incidence


In [ ]:
  def lookback_pattern(df, codes=None, cols=None, pid='pid', date='date', 
            year=None, step=200, pct=False):
    """
    The number (or percent) of new cases in a given year that remain new cases as the length of the look-back period is gradually increased.
    
    Args:
      df: dataframe with information about individual level events
      codes (string or list of str): the codes for the disease you want to count
      cols (str or list of str): the colum(s) where the codes are
      pid (str): the personal identifier used in the dataframe
      year (None, int, or list of int): The year for which you want incidence estimates. Default is None which calculates the incidence for all possible years
      step (int): The stepwise increase in the look-back period (measured in days) used when creating the historical pattern between the size of the lookback period and the share of incidence patients in a year that remain incidence patients as more historical data is added 
      pct (bool, default: False): If True, the function will return an estimate of the patients

  Example:
      w=lookback_pattern(df=df, codes='K50', cols='icd', step=100, pct=True) 
    """
    # if no codes are specified, all observations are relevant 
    # (when the dataframe already has only relevant observations)
    if codes:
        rows=select_rows(df=df, codes=codes, cols=cols, pid=pid)
        df=df[rows]
    
    if (year is None) or isinstance(year,list):
        still_left=_lookback_years(df=df, codes=None, cols=None, pid=pid, 
                                   date=date, year=year, step=step, 
                                   required_events=required_events, pct=pct)
        return still_left
    
    patient_pids=set(df.loc[df[date].dt.year==year, pid].unique())
    n_patients=len(patient_pids)
    
    historical_df=df[(df.pid.isin(patient_pids)) & (df[date].dt.year<year)]

    start_date=pd.to_datetime(f'01-01-{year}')
    min_date=df[date].min()
    range_days=(start_date-min_date).days
    
    historical_df['days_from_start']=(start_date-historical_df[date]).dt.days
        
    if pct:
        still_first={0:1.0}  
    else:
        still_first={0:n_patients}  
        
    for days in range(step,range_days,step):
        washout_rows=historical_df.days_from_start<days
        washout_pids=set(historical_df.loc[washout_rows,pid].unique())
        with_previous_events_pids=patient_pids.intersection(washout_pids)
        n_with_previous_events= len(with_previous_events_pids)
        n_no_previous_history= n_patients - n_with_previous_events
        if pct:
            still_first[days]= n_no_previous_history/n_patients
        else:
            still_first[days]= n_no_previous_history
    return pd.Series(still_first)

def _lookback_years(df, codes, cols, pid='pid', date='date', 
                   year=None, step=200, required_events=1, pct=False):
  """
  Helper function used when multiple years is used in lookback_pattern()
  """
    if year is None:
        min_year=df[date].min().year
        years=sorted(df.loc[df.date.dt.year>min_year, date].dt.year.unique())
    else:
        years=year
        
    still_left={}
    for year in years:
        still_left[year]=lookback_pattern(df=df, codes=codes, cols=cols, pid=pid, 
                                 date=date, year=year, step=step, 
                                 required_events=required_events, pct=pct)
    return pd.DataFrame(still_left)


In [ ]:
def single_pattern(df, codes=None, cols=None, pid='pid', date='date', 
            year=None, step=0, required_events=1, pct=False):
  """
    The percent unique patients in a given year that remain unique when more data is gradually added before and after the year in question
    
    Args:
      df: dataframe with information about individual level events
      codes (string or list of str): the codes for the disease you want to count
      cols (str or list of str): the colum(s) where the codes are
      pid (str): the personal identifier used in the dataframe
      year (None, int, or list of int): The year for which you want incidence estimates. Default is None which calculates the incidence for all possible years
      step (int): The stepwise increase by which the observed time-period is increased (both before and after the given year).     
      pct (bool, default: False): If True, the function will return an estimate of the patients
  
  Example:
    s=singles(df=df, codes='K50', cols='icd', date='date',step=100,pct=True)         

  Note:
    If the data available before/after the given year is not of equal length, the algorithm will keed adding data in steps until it hits the end of the data at one end - and then stop.

  """
    # if no codes are specified, all observations are relevant 
    # (when the dataframe already has only relevant observations)
    if codes:
        rows=select_rows(df=df, codes=codes, cols=cols, pid=pid)
        df=df[rows]
    
    if (year is None) or isinstance(year,list):
        still_single=_single_years(df=df, codes=codes, cols=cols, pid=pid, date=date, year=year, step=step, required_events=required_events, pct=pct)
        return still_single
    include = (df[date].dt.year==year)
    single_rows = df[include].groupby(pid).size()<=required_events
    original_single_pids = set(single_rows.index[single_rows==1])
    
    n_patients=df.loc[include, pid].nunique()
    min_date=df.date.min()
    max_date=df.date.max()
    
    date_range=(max_date-min_date).days
    still_single={}
    for expand_days in range(0,date_range,step):
        start_date = datetime(year=year, month=1, day=1) - timedelta(days=expand_days)
        end_date = datetime(year=year, month=12, day=31) + timedelta(days=expand_days)
        if (start_date<min_date) or (end_date>max_date):
            break
        larger_sample = (start_date < df[date]) & (df[date] < end_date)
        
        single_rows = df[larger_sample].groupby(pid).size()<=required_events
        new_single_pids = set(single_rows.index[single_rows==1])
        
        still_single_pids = original_single_pids & new_single_pids
        if pct:
            still_single[expand_days] = len(still_single_pids)/n_patients
        else:
            still_single[expand_days] = len(still_single_pids)
    
    return pd.Series(still_single, dtype='float64')

def _single_years(df, codes, cols, pid='pid', date='date', year=None, step=200, required_events=1, pct=False):
    """
    Helper function used when multiple years is used in single_pattern()
    """
    if year is None:
        min_year=df[date].min().year
        years=sorted(df.loc[df.date.dt.year>min_year, date].dt.year.unique())
    else:
        years=year
        
    still_single={}
    for year in years:
        still_single[year]=single_pattern(df=df, codes=codes, cols=cols, pid=pid, date=date, year=year, step=step, required_events=required_events, pct=pct)
    return pd.DataFrame(still_single)

In [ ]:
def fit_curve(df, pred=True, func=False):
  """
  Estimates the coefficient in a function that gives the best fit to the pattern in the data
  
  Args:
    df (dataframe): a dataframe with the data for the pattern that will be fitted
    pred (bool): Will print the predicted values if True
    func (function): a user defined function that defines the curve. If False, default is a hyperbolic curve: a + (b/(1+c*x))

  Returns (array): A list of the estimated parameters
    
  """
  if not func:
    def func(x, a, b, c):
      return  a + (b/(1+c*x))

  df=df.stack().reset_index()
  df.columns=['x', 'year', 'y']
  #does not make sense to have zero history added so it messes up results to include it
  df=df[df['x']!=0]
  coeff, pcov = optimize.curve_fit(func, df.x.to_numpy(), df.y.to_numpy())
  pred_y = [func(x, *coeff) for x in range(0,2000,100) ]
  if pred:
    print(pred_y)
  # consider returing covariances as well? (pcov)
  return coeff


In [ ]:
def adjusted_incidence(df, codes=None, cols=None, pid='pid', date='date', required_events=1, start_year=None, 
            year=None, step=0, lookback_func=False, single_func=False):
    # if no codes are specified, all observations are counted as relevant for the incidence
    """
    The number of new cases each year after adjusting for lookback and lookahead  

    Args:
      df: dataframe with information about individual level events
      codes (string or list of str): the codes for the disease you want to count
      cols (str or list of str): the colum(s) where the codes are
      pid (str): the personal identifier used in the dataframe
      year (None, int, or list of int): The year for which you want incidence estimates. Default is None which calculates the incidence for all possible years
      step (int): The stepwise increase in the look-back period (measured in days) used when creating the historical pattern between the size of the lookback period and the share of incidence patients in a year that remain incidence patients as more historical data is added 
      start_year (int): the first year for which you will provide incidence estimates
      lookback_func (func): the function that describes the lookback pattern as more data is added (default is hyperbolic)
      single_func (func): the function that describes the pattern of unique cases as more data is added (default is hyperbolic)

    Returns:
      A Series with estimates of the number of new cases each year after adjusting for lookback and lookforward based on the trends in the observed data 
    
    Example:
      n=adjusted_incidence(df=df, codes='K50', cols='icd', date='date',step=50, start_year=2008)

    Note:
      If user supplied functions are used the first coefficient must be the limit of the curves (constant)         

    """
     
    if codes:
        rows=select_rows(df=df, codes=codes, cols=cols, pid=pid)
        df=df[rows]    

    #adjust for washout
    raw=raw_incidence(df=df, date=date)  
    
    washout_pattern=lookback_pattern(df=df, step=step, pct=True)

    if not lookback_func:
      def lookback_func(x, a, b, c):
          return  a + (b/(1+c*x))         
    
    washout_curve=fit_curve(df=washout_pattern, func=lookback_func)
    
    # start from first year of data with one year of lookback
    lookback_days=(raw.index-start_year)*365    
    pred_y = [func(x, *washout_curve) for x in lookback_days ]
    adjustment_factor = washout_curve[0]/pred_y
    adjusted_for_washout = raw * adjustment_factor
    
    adjusted_incidence = adjusted_for_washout
    
    # forward bias
    if required_events==2:

      df['__year']=df[date].dt.year
      
      n_patients=df.groupby('__year').pid.nunique()
      
      forward=single_pattern(df=df, step=step, pct=True)         
      
      if not single_func:
        def single_func(x, a, b, c):
            return  a + (b/(1+c*x))
      single_curve=fit_curve(df=forward, func=single_func)
      n_true_singles=n_patients*single_curve[0]
      
      adjusted_incidence=adjusted_for_washout-n_true_singles
    elif required_events>2:
      print('Adjustment when more than 2 events is required is not yet implemented')
      adjusted_incidence=None
    return adjusted_incidence

